In [ ]:
!curl -ssL https://magic.modular.com/ | bash

In [ ]:
import os
os.environ['PATH'] += ':/root/.modular/bin'

In [ ]:
!magic init mojo_kernels --format mojoproject

In [ ]:
%cd mojo_kernels/

In [38]:
%%writefile tensor_indexer_kernel.mojo

from gpu import block_idx, thread_idx
from gpu.host import DeviceContext
from math import iota
from layout import Layout, LayoutTensor

alias dtype = DType.uint32
alias blocks = 4
alias threads = 4
var elems_count = blocks * threads
alias layout = Layout.row_major(blocks, threads)
#ImmutableAnyOrigin/MutableAnyOrigin not making any difference
#alias InputTensor = LayoutTensor[dtype, layout, ImmutableAnyOrigin]
alias InputTensor = LayoutTensor[dtype, layout, MutableAnyOrigin]

fn tensor_indexer_kernel(input_tensor: InputTensor):
    var bid = block_idx.x
    var tid = thread_idx.x
    input_tensor[bid, tid] *= 3
    print("Block id: ", bid, ", thread id: ", tid, ", value: ", input_tensor[bid, tid])


def main():
    var ctx = DeviceContext()
    var device_input_buff = ctx.enqueue_create_buffer[dtype](elems_count)

    with device_input_buff.map_to_host() as host_input_buff:
      iota(host_input_buff.unsafe_ptr(), elems_count)
      print(host_input_buff)

    var input_tensor = InputTensor(device_input_buff)
    ctx.enqueue_function[tensor_indexer_kernel](input_tensor, grid_dim=blocks, block_dim=threads)
    ctx.synchronize()

    with device_input_buff.map_to_host() as host_input_buff:
      print(host_input_buff)
      #tensor = InputTensor(host_input_buff)
      tensor1 = LayoutTensor[dtype, Layout.col_major(2, 8)](host_input_buff)
      print(tensor1)
      tensor2 = LayoutTensor[dtype, Layout.col_major(8, 2)](host_input_buff)
      print(tensor2)

      tensor3 = LayoutTensor[dtype, Layout.row_major(8, 2)](host_input_buff)
      print(tensor3)
      print(host_input_buff)




Overwriting tensor_indexer_kernel.mojo


In [39]:
!magic run mojo tensor_indexer_kernel.mojo

HostBuffer([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
Block id:  2 , thread id:  0 , value:  24
Block id:  2 , thread id:  1 , value:  27
Block id:  2 , thread id:  2 , value:  30
Block id:  2 , thread id:  3 , value:  33
Block id:  0 , thread id:  0 , value:  0
Block id:  0 , thread id:  1 , value:  3
Block id:  0 , thread id:  2 , value:  6
Block id:  0 , thread id:  3 , value:  9
Block id:  1 , thread id:  0 , value:  12
Block id:  1 , thread id:  1 , value:  15
Block id:  1 , thread id:  2 , value:  18
Block id:  1 , thread id:  3 , value:  21
Block id:  3 , thread id:  0 , value:  36
Block id:  3 , thread id:  1 , value:  39
Block id:  3 , thread id:  2 , value:  42
Block id:  3 , thread id:  3 , value:  45
HostBuffer([0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45])
0 6 12 18 24 30 36 42 
3 9 15 21 27 33 39 45 
0 24 
3 27 
6 30 
9 33 
12 36 
15 39 
18 42 
21 45 
0 3 
6 9 
12 15 
18 21 
24 27 
30 33 
36 39 
42 45 
HostBuffer([0, 3, 6, 9, 12, 15, 18, 21, 24